In [29]:
import sqlite3
from pathlib import Path
import pandas as pd
import numpy as np
from nba_api.stats.endpoints import leaguegamelog, playergamelogs, teamgamelogs
from time import sleep
import time
from typing import Dict, Tuple
from tqdm import tqdm

from autogluon.tabular import TabularPredictor, TabularDataset
from autogluon.common import space




In [30]:
DB_NAME = Path.cwd().parent / 'data' / 'nba_stats.db'

print(DB_NAME)

query = """ SELECT
    tbs.SEASON_YEAR,
    tbs.TEAM_ID,
    tbs.TEAM_ABBREVIATION,
    tbs.GAME_ID,
    tbs.GAME_DATE,
    tbs.MATCHUP,
    tbs.WL,
    tbs.MIN,
    tbs.PTS,
    tbs.FGM,
    tbs.FGA,
    tbs.FG3M,
    tbs.FG3A,
    tbs.FTM,
    tbs.FTA,
    tbs.OREB,
    tbs.DREB,
    tbs.REB,
    tbs.AST,
    tbs.TOV,
    tbs.STL,
    tbs.BLK,
    tbs.PLUS_MINUS,
    tas.OFF_RATING,
    tas.DEF_RATING,
    tas.NET_RATING,
    tas.PACE,
    tas.PIE,
    tss.PCT_PTS_2PT_MR,
    tss.PCT_AST_2PM,
    tss.PCT_UAST_2PM,
    tss.PCT_AST_3PM,
    tss.PCT_UAST_3PM,
    ths.defensiveBoxOuts as DEFENSIVE_BOX_OUTS,
    ths.contestedShots2pt as CONTESTED_SHOTS_2PT,
    ths.contestedShots3pt as CONTESTED_SHOTS_3PT,
    ths.DEFLECTIONS,
    tts.DIST,
    tts.TCHS,
    tts.PASS,
    tts.CFGM,
    tts.CFGA,
    tts.UFGM,
    tts.UFGA,
    tts.DFGM,
    tts.DFGA,
    dsbp.OPP_FGM_C,
    dsbp.OPP_FGA_C,
    dsbp.OPP_FGM_F,
    dsbp.OPP_FGA_F,
    dsbp.OPP_FGM_G,
    dsbp.OPP_FGA_G,
    dsbp.OPP_FG3M_C,
    dsbp.OPP_FG3A_C,
    dsbp.OPP_FG3M_F,
    dsbp.OPP_FG3A_F,
    dsbp.OPP_FG3M_G,
    dsbp.OPP_FG3A_G,
    dsbp.OPP_FTM_C,
    dsbp.OPP_FTM_F,
    dsbp.OPP_FTM_G,
    dsbp.OPP_FTA_C,
    dsbp.OPP_FTA_F,
    dsbp.OPP_FTA_G,
    tms.PTS_OFF_TOV,
    tms.PTS_2ND_CHANCE,
    tms.PTS_FB,
    tms.PTS_PAINT,
    tms.OPP_PTS_OFF_TOV,
    tms.OPP_PTS_2ND_CHANCE,
    tms.OPP_PTS_FB,
    tms.OPP_PTS_PAINT,
    tms.BLK as BLOCKS,
    tms.BLKA as BLOCKED_ATT,
    tms.PF,
    tms.PFD,
    tslb.[Restricted Area FGM] as FGM_RESTRICTED,
    tslb.[Restricted Area FGA] as FGA_RESTRICTED,
    tslb.[In The Paint (Non-RA) FGM] as FGM_PAINT_NON_RA,
    tslb.[In The Paint (Non-RA) FGA] as FGA_PAINT_NON_RA,
    tslb.[Mid-Range FGM] as FGM_MIDRANGE,
    tslb.[Mid-Range FGA] as FGA_MIDRANGE,
    tslb.[Corner 3 FGM] as FGM_CORNER3,
    tslb.[Corner 3 FGA] as FGA_CORNER3,
    tslb.[Above the Break 3 FGM] as FGM_ABOVE_BREAK3,
    tslb.[Above the Break 3 FGA] as FGA_ABOVE_BREAK3,
    CASE WHEN MATCHUP like '%@%' THEN 0 ELSE 1 END as IS_HOME
FROM team_basic_stats tbs
LEFT JOIN team_advanced_stats tas ON tbs.TEAM_ID = tas.TEAM_ID AND tbs.GAME_ID = tas.GAME_ID
LEFT JOIN team_scoring_stats tss ON tbs.TEAM_ID = tss.TEAM_ID AND tbs.GAME_ID = tss.GAME_ID
LEFT JOIN team_hustle_stats ths ON tbs.TEAM_ID = ths.teamId AND tbs.GAME_ID = ths.gameId
LEFT JOIN team_track_stats tts ON tbs.TEAM_ID = tts.TEAM_ID AND tbs.GAME_ID = tts.GAME_ID
LEFT JOIN defensive_stats_by_position dsbp ON tbs.TEAM_ID = dsbp.TEAM_ID AND tbs.GAME_DATE = dsbp.GAME_DATE
LEFT JOIN team_miscellaneous_stats tms ON tbs.TEAM_ID = tms.TEAM_ID AND tbs.GAME_ID = tms.GAME_ID
LEFT JOIN team_shot_location_boxscores tslb ON tbs.TEAM_ID = tslb.TEAM_ID AND tbs.GAME_DATE = tslb.GAME_DATE
WHERE tbs.SEASON_YEAR BETWEEN '2016-17' AND '2024-25'
ORDER BY tbs.GAME_DATE, tbs.TEAM_ID"""

conn = sqlite3.connect(DB_NAME)
df_team = pd.read_sql(query, conn)
conn.close()


c:\Users\Jordan Nishimura\nba_model_using_player_embeddings\data\nba_stats.db


In [31]:

def clean_team_data(df):
    """This function cleans the team_data
    1) Changes W/L to 1/0 
    2) Changes franchise abbreviations to their most 
    recent abbreviation for consistency
    3) Converts GAME_DATE to datetime object
    4) Creates a binary column 'HOME_GAME'
    5) Removes 3 games where advanced stats were not collected
    """
    df = df.copy()
    df['WL'] = (df['WL'] == 'W').astype(int)
    df = df.rename(columns={'SEASON_YEAR': 'SEASON'})

    abbr_mapping = {'NJN': 'BKN',
                    'CHH': 'CHA',
                    'VAN': 'MEM',
                    'NOH': 'NOP',
                    'NOK': 'NOP',
                    'SEA': 'OKC'}

    df['TEAM_ABBREVIATION'] = df['TEAM_ABBREVIATION'].replace(abbr_mapping)
    df['MATCHUP'] = df['MATCHUP'].str.replace('NJN', 'BKN')
    df['MATCHUP'] = df['MATCHUP'].str.replace('CHH', 'CHA')
    df['MATCHUP'] = df['MATCHUP'].str.replace('VAN', 'MEM')
    df['MATCHUP'] = df['MATCHUP'].str.replace('NOH', 'NOP')
    df['MATCHUP'] = df['MATCHUP'].str.replace('NOK', 'NOP')
    df['MATCHUP'] = df['MATCHUP'].str.replace('SEA', 'OKC')

    df['GAME_DATE'] = pd.to_datetime(df['GAME_DATE'])
    
    df = df.fillna(0)
    
    return df


df_team_clean = clean_team_data(df_team)



In [32]:


def prep_for_aggregation_team(df):
    """This function...
    1) Removes categories that are percentages,
    as we will be averaging them and do not want to average 
    percentages. 
    2) Converts shooting percentage stats into raw values"""

    df['FG2M'] = df['FGM'] - df['FG3M']
    df['FG2A'] = df['FGA'] - df['FG3A']
    df['PTS_2PT_MR'] = (df['PTS'] * df['PCT_PTS_2PT_MR']) #.astype('int8')
    df['AST_2PM'] = (df['FG2M'] * df['PCT_AST_2PM']) #.astype('int8')
    df['AST_3PM'] = (df['FG3M'] * df['PCT_AST_3PM']) #.astype('int8')
    df['UAST_2PM'] = (df['FG2M'] * df['PCT_UAST_2PM']) #.astype('int8')
    df['UAST_3PM'] = (df['FG3M'] * df['PCT_UAST_3PM']) #.astype('int8')

    df['OPP_FG2M_G'] = df['OPP_FGM_G'] - df['OPP_FG3M_G']
    df['OPP_FG2A_G'] = df['OPP_FGA_G'] - df['OPP_FG3A_G']

    df['OPP_FG2M_F'] = df['OPP_FGM_F'] - df['OPP_FG3M_F']
    df['OPP_FG2A_F'] = df['OPP_FGA_F'] - df['OPP_FG3A_F']
    
    df['OPP_FG2M_C'] = df['OPP_FGM_C'] - df['OPP_FG3M_C']
    df['OPP_FG2A_C'] = df['OPP_FGA_C'] - df['OPP_FG3A_C']

    df['POINT_DIFF'] = df['PLUS_MINUS']
    df['RECORD'] = df['WL']
    df['TEAM_SCORE'] = df['PTS']
    
    # percentage_columns = [x for x in df.columns if 'PCT' in x]
    drop_cols= ['OPP_FGM_G', 'OPP_FGA_G', 'OPP_FGM_F', 'OPP_FGA_F', 'OPP_FGM_C', 'OPP_FGA_C']
    #                       'MIN', 'PIE', 'PIE']
        
    df = df.drop(columns = drop_cols)
    
    ## Reorder Columns


    return df


df_team_clean2 = prep_for_aggregation_team(df_team_clean)



In [33]:


def create_matchups(df):
    """This function makes each row a matchup between 
    team and opp"""
    keep_cols = ['SEASON', 'TEAM_ID', 'TEAM_ABBREVIATION', 'GAME_ID', 'GAME_DATE', 'MATCHUP', 'WL']
    stat_cols = [x for x in df.columns if x not in keep_cols]
    

    
    matchups = pd.merge(df, df, how='left', on=['GAME_ID'], suffixes=['', '_opp'])
    matchups = matchups.loc[matchups['TEAM_ID'] != matchups['TEAM_ID_opp']]

    matchups = matchups.drop(columns = ['SEASON_opp', 'TEAM_ID_opp', 'TEAM_ABBREVIATION_opp', 'GAME_DATE_opp',
                                         'MATCHUP_opp', 'TEAM_ID_opp', 'WL_opp']
                 )
    
    matchups
    
    return matchups


matchups = create_matchups(df_team_clean2)


team_optimal_spans = {"PTS": 29,
    "FGM": 33,
    "FGA": 32,
    "FG3M": 28,
    "FG3A": 19,
    "FTM": 37,
    "FTA": 37,
    "OREB": 33,
    "DREB": 37,
    "REB": 33,
    "AST": 30,
    "TOV": 39,
    "STL": 36,
    "BLK": 35,
    "PLUS_MINUS": 38,
    "OFF_RATING": 37,
    "DEF_RATING": 43,
    "NET_RATING": 39,
    "PACE": 17,
    "PIE": 37,
    "PCT_PTS_2PT_MR": 20,
    "PCT_AST_2PM": 31,
    "PCT_UAST_2PM": 31,
    "PCT_AST_3PM": 37,
    "PCT_UAST_3PM": 38,
    "DEFENSIVE_BOX_OUTS": 11,
    "CONTESTED_SHOTS_2PT": 24,
    "CONTESTED_SHOTS_3PT": 25,
    "deflections": 30,
    "DIST": 43,
    "TCHS": 31,
    "PASS": 22,
    "CFGM": 29,
    "CFGA": 26,
    "UFGM": 34,
    "UFGA": 32,
    "DFGM": 31,
    "DFGA": 29,
    "OPP_FG3M_C": 10,
    "OPP_FG3A_C": 7,
    "OPP_FG3M_F": 18,
    "OPP_FG3A_F": 11,
    "OPP_FG3M_G": 19,
    "OPP_FG3A_G": 12,
    "OPP_FTM_C": 12,
    "OPP_FTM_F": 20,
    "OPP_FTM_G": 21,
    "OPP_FTA_C": 11,
    "OPP_FTA_F": 18,
    "OPP_FTA_G": 19,
    "PTS_OFF_TOV": 45,
    "PTS_2ND_CHANCE": 37,
    "PTS_FB": 31,
    "PTS_PAINT": 27,
    "OPP_PTS_OFF_TOV": 47,
    "OPP_PTS_2ND_CHANCE": 47,
    "OPP_PTS_FB": 52,
    "OPP_PTS_PAINT": 32,
    "BLOCKS": 35,
    "BLOCKED_ATT": 40,
    "PF": 33,
    "PFD": 29,
    "FGM_RESTRICTED": 25,
    "FGA_RESTRICTED": 24,
    "FGM_PAINT_NON_RA": 35,
    "FGA_PAINT_NON_RA": 27,
    "FGM_MIDRANGE": 21,
    "FGA_MIDRANGE": 16,
    "FGM_CORNER3": 44,
    "FGA_CORNER3": 35,
    "FGM_ABOVE_BREAK3": 31,
    "FGA_ABOVE_BREAK3": 21,
    "FG2M": 26,
    "FG2A": 22,
    "PTS_2PT_MR": 21,
    "AST_2PM": 24,
    "AST_3PM": 31,
    "UAST_2PM": 31,
    "UAST_3PM": 36,
    "OPP_FG2M_G": 11,
    "OPP_FG2A_G": 9,
    "OPP_FG2M_F": 9,
    "OPP_FG2A_F": 8,
    "OPP_FG2M_C": 7,
    "OPP_FG2A_C": 6,
    "PTS_opp": 32,
    "FGM_opp": 34,
    "FGA_opp": 28,
    "FG3M_opp": 36,
    "FG3A_opp": 28,
    "FTM_opp": 43,
    "FTA_opp": 41,
    "OREB_opp": 50,
    "DREB_opp": 39,
    "REB_opp": 39,
    "AST_opp": 40,
    "TOV_opp": 34,
    "STL_opp": 41,
    "BLK_opp": 40,
    "PLUS_MINUS_opp": 38,
    "OFF_RATING_opp": 43,
    "DEF_RATING_opp": 37,
    "NET_RATING_opp": 39,
    "PACE_opp": 17,
    "PIE_opp": 36,
    "PCT_PTS_2PT_MR_opp": 35,
    "PCT_AST_2PM_opp": 43,
    "PCT_UAST_2PM_opp": 43,
    "PCT_AST_3PM_opp": 44,
    "PCT_UAST_3PM_opp": 44,
    "DEFENSIVE_BOX_OUTS_opp": 10,
    "CONTESTED_SHOTS_2PT_opp": 22,
    "CONTESTED_SHOTS_3PT_opp": 21,
    "deflections_opp": 36,
    "DIST_opp": 48,
    "TCHS_opp": 41,
    "PASS_opp": 39,
    "CFGM_opp": 36,
    "CFGA_opp": 33,
    "UFGM_opp": 44,
    "UFGA_opp": 36,
    "DFGM_opp": 26,
    "DFGA_opp": 24,
    "OPP_FG3M_C_opp": 52,
    "OPP_FG3A_C_opp": 50,
    "OPP_FG3M_F_opp": 36,
    "OPP_FG3A_F_opp": 30,
    "OPP_FG3M_G_opp": 33,
    "OPP_FG3A_G_opp": 30,
    "OPP_FTM_C_opp": 44,
    "OPP_FTM_F_opp": 40,
    "OPP_FTM_G_opp": 46,
    "OPP_FTA_C_opp": 44,
    "OPP_FTA_F_opp": 40,
    "OPP_FTA_G_opp": 42,
    "PTS_OFF_TOV_opp": 47,
    "PTS_2ND_CHANCE_opp": 47,
    "PTS_FB_opp": 52,
    "PTS_PAINT_opp": 32,
    "OPP_PTS_OFF_TOV_opp": 45,
    "OPP_PTS_2ND_CHANCE_opp": 37,
    "OPP_PTS_FB_opp": 31,
    "OPP_PTS_PAINT_opp": 27,
    "BLOCKS_opp": 40,
    "BLOCKED_ATT_opp": 35,
    "PF_opp": 29,
    "PFD_opp": 33,
    "FGM_RESTRICTED_opp": 31,
    "FGA_RESTRICTED_opp": 30,
    "FGM_PAINT_NON_RA_opp": 52,
    "FGA_PAINT_NON_RA_opp": 51,
    "FGM_MIDRANGE_opp": 40,
    "FGA_MIDRANGE_opp": 30,
    "FGM_CORNER3_opp": 48,
    "FGA_CORNER3_opp": 36,
    "FGM_ABOVE_BREAK3_opp": 46,
    "FGA_ABOVE_BREAK3_opp": 35,
    "FG2M_opp": 31,
    "FG2A_opp": 26,
    "PTS_2PT_MR_opp": 40,
    "AST_2PM_opp": 37,
    "AST_3PM_opp": 38,
    "UAST_2PM_opp": 35,
    "UAST_3PM_opp": 52,
    "OPP_FG2M_G_opp": 40,
    "OPP_FG2A_G_opp": 34,
    "OPP_FG2M_F_opp": 34,
    "OPP_FG2A_F_opp": 33,
    "OPP_FG2M_C_opp": 52,
    "OPP_FG2A_C_opp": 49
}


def apply_optimal_spans(df: pd.DataFrame, optimal_spans: dict, grouping_col: str) -> pd.DataFrame:
    # Create a copy of the input DataFrame to avoid modifying the original
    df = df.copy()
    
    # Sort the DataFrame
    df = df.sort_values([grouping_col, 'GAME_DATE'])
    
    # Pre-calculate all EWA columns at once
    ewa_columns = {}
    
    for feature, span in optimal_spans.items():
        grouped = df.groupby(grouping_col)[feature]
        ewa_values = grouped.transform(
            lambda x: x.ewm(span=span, adjust=False).mean().shift(1)
        )
        ewa_columns[f'{feature}_EWA'] = ewa_values
    
    # Combine all new columns at once using concat
    result = pd.concat([df, pd.DataFrame(ewa_columns)], axis=1)
    
    return result


df_team_with_ewa = apply_optimal_spans(matchups, team_optimal_spans, 'TEAM_ID')


In [34]:


def add_percentage_features(df):
    """Add the following features for both team and opp:
    OREB_PCT, DREB_PCT, REB_PCT, TS_PCT, EFG_PCT, AST_RATIO, TOV_PCT, PIE.
    """
    
    df = df.copy()
    
    df['FG2_PCT'] = df['FG2M'] / df[f'FG2A']
    df['FG3_PCT'] = df['FG3M'] / df[f'FG3A']
    
    df['OREB_PCT'] = df['OREB'] / (df['OREB'] + df['DREB_opp'])
    df['OREB_PCT_opp'] = df['OREB_opp'] / (df['OREB_opp'] + df['DREB'])

    df['DREB_PCT'] = df['DREB'] / (df['DREB'] + df['OREB_opp'])
    df['DREB_PCT_opp'] = df['DREB_opp'] / (df['DREB_opp'] + df['OREB'])

    df['REB_PCT'] = df['REB'] / (df['REB'] + df['REB_opp'])
    df['REB_PCT_opp'] = df['REB_opp'] / (df['REB_opp'] + df['REB'])

    df['TS_PCT'] = df['PTS'] / ((2*(df['FG2A'] + df['FG3A']) + 0.44*df['FTA']))
    
    df['TS_PCT_opp'] = df['PTS_opp'] / ((2*(df['FG2A_opp'] + df['FG3A_opp']) + 0.44*df['FTA_opp']))

    df['EFG_PCT'] = (df['FG2M'] + 1.5*df['FG3M']) / (df['FG2A']
                                                                    + df['FG3A'])
    df['EFG_PCT_opp'] = (df['FG2M_opp'] + 1.5*df['FG3M_opp']) / (df['FG2A_opp'] 
                                                                 + df['FG3A_opp'])

    df['AST_RATIO'] = (df['AST'] * 100) / df['PACE']
    df['AST_RATIO_opp'] = (df['AST_opp'] * 100) / df['PACE_opp']

    df['TOV_PCT'] = 100*df['TOV'] / (df['FG2A'] 
                                               + df['FG3A'] 
                                               + 0.44*df['FTA'] 
                                               + df['TOV'])
    
    df['TOV_PCT_opp'] = 100*df['TOV_opp'] / (df['FG2A_opp'] 
                                             + df['FG3A_opp'] 
                                             + 0.44*df['FTA_opp'] 
                                             + df['TOV_opp'])
    
    
    df['PIE'] = ((df['PTS'] + df['FG2M'] + df['FG3M'] + df['FTM'] 
                 - df['FG2A'] - df['FG3A'] - df['FTA'] 
                 + df['DREB'] + df['OREB']/2
                + df['AST'] + df['STL'] + df['BLK']/2
                - df['PF'] - df['TOV']) 
                 / (df['PTS'] + df['PTS_opp'] + df['FG2M'] + df['FG2M_opp']
                   + df['FG3M'] + df['FG3M_opp'] + df['FTM'] + df['FTM_opp']
                   - df['FG2A'] - df['FG2A_opp'] - df['FG3A'] - df['FG3A_opp'] 
                    - df['FTA'] - df['FTA_opp'] + df['DREB'] + df['DREB_opp']
                    + (df['OREB']+df['OREB_opp'])/2 + df['AST'] + df['AST_opp']
                    + df['STL'] + df['STL_opp'] + (df['BLK'] + df['BLK_opp'])/2
                    - df['PF'] - df['PF_opp'] - df['TOV'] - df['TOV_opp']))
        
    return df
  
  
  
df_team_with_ewa2 = add_percentage_features(df_team_with_ewa)



In [35]:


def add_schedule_features(df: pd.DataFrame) -> pd.DataFrame:
    """Add features related to rest and schedule density"""
    df = df.copy()
    df = df.sort_values(['TEAM_ID', 'GAME_DATE'])

    # Calculate days between games for teams
    df['DAYS_REST'] = df.groupby('TEAM_ID')['GAME_DATE'].diff().dt.days
    
    # Fill NaN values for first games
    df['DAYS_REST'] = df['DAYS_REST'].fillna(5)
    
    
    return df


df_team_with_ewa3 = add_schedule_features(df_team_with_ewa2)



In [36]:


def create_matchup_df2(df):
    df_team_merged = pd.merge(df, df, on=['GAME_ID', 'GAME_DATE'], suffixes=('_tm', '_opp'))
    df_team_merged = df_team_merged.loc[df_team_merged['TEAM_ID_tm'] != df_team_merged['TEAM_ID_opp']]
    df_team_merged = df_team_merged.sort_values(['GAME_DATE', 'GAME_ID', 'IS_HOME_tm'])

    keep_cols = ['SEASON_tm', 'TEAM_ID_tm', 'TEAM_ABBREVIATION_tm', 'TEAM_ID_opp', 'TEAM_ABBREVIATION_opp',
                 'GAME_ID', 'GAME_DATE', 'MATCHUP_tm', 'IS_HOME_tm', 'WL_tm', 'PTS_tm', 'PTS_opp', 'DAYS_REST_tm', 'DAYS_REST_opp']
    ewa_cols = [col for col in df_team_merged.columns if "EWA" in col]
    keep_cols.extend(ewa_cols)
    
    df_team_merged = df_team_merged.rename(columns={'EWA_tm': 'EWA_tm_tm', 'EWA_opp': 'EWA_tm_opp'})
    
    return df_team_merged[keep_cols] 


df_matchup_team = create_matchup_df2(df_team_with_ewa3)



In [37]:


final_df = df_matchup_team.dropna()


final_df = final_df.rename(columns ={'SEASON_tm':'SEASON'
                                     ,'TEAM_ID_tm':'TEAM_ID_tm'
                                     ,'TEAM_ABBREVIATION_tm':'TEAM_ABBREVIATION'
                                     ,'MATCHUP_tm':'MATCHUP'})

final_train = final_df[final_df['SEASON'] <= '2023-24']
final_test = final_df[final_df['SEASON'] == '2024-25']

model_path = Path.cwd().parent / "models" / "AutogluonModels" / "ag-20241116_093710"
print("model_path:", model_path)

# Prepare the features
ewa_columns = [col for col in final_df.columns if '_EWA' in col]
ewa_columns.extend(['IS_HOME_tm', 'DAYS_REST_tm', 'DAYS_REST_opp'])

X_train = final_train[ewa_columns]
y_train = final_train['PTS_tm']

X_test = final_test[ewa_columns]
y_test = final_test['PTS_tm']

predictor = TabularPredictor.load(path=model_path)

# First get predictions on test data
predictions = predictor.predict(X_test)

# Create DataFrame with actual vs predicted scores
results_df = pd.DataFrame({
    'Team': final_test['TEAM_ABBREVIATION'],
    'Opponent': final_test['TEAM_ABBREVIATION_opp'],
    'GAME_DATE':final_test['GAME_DATE'],
    'GAME_ID': final_test['GAME_ID'],
    'Matchup': final_test['MATCHUP'],
    'Home': final_test['IS_HOME_tm'],
    'Actual_Score': y_test,
    'Predicted_Score': predictions
})

results_df.to_csv('prediction_results.csv', index=False)

model_path: c:\Users\Jordan Nishimura\nba_model_using_player_embeddings\models\AutogluonModels\ag-20241116_093710


In [38]:
X_train.columns.tolist()

['PTS_EWA_tm',
 'FGM_EWA_tm',
 'FGA_EWA_tm',
 'FG3M_EWA_tm',
 'FG3A_EWA_tm',
 'FTM_EWA_tm',
 'FTA_EWA_tm',
 'OREB_EWA_tm',
 'DREB_EWA_tm',
 'REB_EWA_tm',
 'AST_EWA_tm',
 'TOV_EWA_tm',
 'STL_EWA_tm',
 'BLK_EWA_tm',
 'PLUS_MINUS_EWA_tm',
 'OFF_RATING_EWA_tm',
 'DEF_RATING_EWA_tm',
 'NET_RATING_EWA_tm',
 'PACE_EWA_tm',
 'PIE_EWA_tm',
 'PCT_PTS_2PT_MR_EWA_tm',
 'PCT_AST_2PM_EWA_tm',
 'PCT_UAST_2PM_EWA_tm',
 'PCT_AST_3PM_EWA_tm',
 'PCT_UAST_3PM_EWA_tm',
 'DEFENSIVE_BOX_OUTS_EWA_tm',
 'CONTESTED_SHOTS_2PT_EWA_tm',
 'CONTESTED_SHOTS_3PT_EWA_tm',
 'deflections_EWA_tm',
 'DIST_EWA_tm',
 'TCHS_EWA_tm',
 'PASS_EWA_tm',
 'CFGM_EWA_tm',
 'CFGA_EWA_tm',
 'UFGM_EWA_tm',
 'UFGA_EWA_tm',
 'DFGM_EWA_tm',
 'DFGA_EWA_tm',
 'OPP_FG3M_C_EWA_tm',
 'OPP_FG3A_C_EWA_tm',
 'OPP_FG3M_F_EWA_tm',
 'OPP_FG3A_F_EWA_tm',
 'OPP_FG3M_G_EWA_tm',
 'OPP_FG3A_G_EWA_tm',
 'OPP_FTM_C_EWA_tm',
 'OPP_FTM_F_EWA_tm',
 'OPP_FTM_G_EWA_tm',
 'OPP_FTA_C_EWA_tm',
 'OPP_FTA_F_EWA_tm',
 'OPP_FTA_G_EWA_tm',
 'PTS_OFF_TOV_EWA_tm',
 'PTS_2

In [39]:
conn = sqlite3.connect(DB_NAME)
tables = pd.read_sql("""select * from sqlite_master where type='table'""", conn)
conn.close
table_list = tables['name'].tolist()
print(table_list)
betting_data = pd.read_sql("SELECT * FROM nba_odds", conn)

conn.close()

betting_data.columns

['traditional_stats', 'advanced_stats', 'scoring_stats', 'defensive_stats', 'hustle_stats', 'player_traditional_stats', 'team_traditional_stats', 'player_advanced_stats', 'team_advanced_stats', 'player_scoring_stats', 'team_scoring_stats', 'player_defensive_stats', 'team_defensive_stats', 'player_hustle_stats', 'team_hustle_stats', 'player_track_stats', 'team_track_stats', 'player_misc_stats', 'team_misc_stats', 'player_usage_stats', 'team_usage_stats', 'player_miscellaneous_stats', 'team_miscellaneous_stats', 'team_basic_stats', 'player_basic_stats', 'player_info', 'team_basic_stats_by_position', 'team_advanced_stats_by_position', 'team_scoring_stats_by_position', 'player_shot_locations_boxscores', 'team_base_stats_by_position', 'defensive_stats_by_position_backup', 'defensive_stats_by_position', 'team_shot_location_boxscores', 'nba_odds', 'team_aggregated_stats']


Index(['GAME_DATE', 'status', 'home_team', 'home_team_loc', 'home_team_abbr',
       'home_team_rank', 'away_team', 'away_team_loc', 'away_team_abbr',
       'away_team_rank', 'home_score', 'away_score', 'home_spread_betmgm',
       'home_spread_fanduel', 'home_spread_caesars', 'home_spread_bet365',
       'home_spread_draftkings', 'home_spread_bet_rivers_ny',
       'home_spread_odds_betmgm', 'home_spread_odds_fanduel',
       'home_spread_odds_caesars', 'home_spread_odds_bet365',
       'home_spread_odds_draftkings', 'home_spread_odds_bet_rivers_ny',
       'away_spread_betmgm', 'away_spread_fanduel', 'away_spread_caesars',
       'away_spread_bet365', 'away_spread_draftkings',
       'away_spread_bet_rivers_ny', 'away_spread_odds_betmgm',
       'away_spread_odds_fanduel', 'away_spread_odds_caesars',
       'away_spread_odds_bet365', 'away_spread_odds_draftkings',
       'away_spread_odds_bet_rivers_ny', 'home_ml_betmgm', 'home_ml_fanduel',
       'home_ml_caesars', 'home_ml_bet365'

In [40]:
np.mean(np.abs(results_df['Predicted_Score'] - results_df['Actual_Score']))

9.655104472313399

In [41]:
betting_data_dk = betting_data[['GAME_DATE', 'status', 'home_team', 'home_team_loc', 'home_team_abbr',
       'home_team_rank', 'away_team', 'away_team_loc', 'away_team_abbr',
       'away_team_rank', 'home_score', 'away_score', 'home_spread_draftkings',
       'home_spread_odds_draftkings', 'away_spread_draftkings', 'away_spread_odds_draftkings', 'home_ml_draftkings',
       'away_ml_draftkings', 'total_draftkings', 'over_odds_draftkings', 'under_odds_draftkings', 'SEASON']]

betting_data_dk = betting_data_dk.rename(columns={'home_team_abbr': 'HOME_TEAM_ABBREVIATION', 'away_team_abbr': 'AWAY_TEAM_ABBREVIATION'})
abbr_dict = {'BK' :'BKN',
'GS' :'GSW',
'NO' :'NOP',
'NY' :'NYK',
'PHO':'PHX',
'SA' :'SAS',}
betting_data_dk['HOME_TEAM_ABBREVIATION'] = betting_data_dk['HOME_TEAM_ABBREVIATION'].replace(abbr_dict)
betting_data_dk['AWAY_TEAM_ABBREVIATION'] = betting_data_dk['AWAY_TEAM_ABBREVIATION'].replace(abbr_dict)
betting_data_dk['GAME_DATE'] = pd.to_datetime(betting_data_dk['GAME_DATE'])
betting_data_dk

,GAME_DATE,status,home_team,home_team_loc,HOME_TEAM_ABBREVIATION,home_team_rank,away_team,away_team_loc,AWAY_TEAM_ABBREVIATION,away_team_rank,...,home_spread_draftkings,home_spread_odds_draftkings,away_spread_draftkings,away_spread_odds_draftkings,home_ml_draftkings,away_ml_draftkings,total_draftkings,over_odds_draftkings,under_odds_draftkings,SEASON
0,2019-10-23,Final OT,Toronto Raptors,Toronto,TOR,-1,New Orleans Pelicans,New Orleans,NOP,-1,...,-1.5,-121.0,1.5,-108.0,-5000.0,1500.0,232.5,-115.0,-112.0,2019-20
1,2019-10-23,Final,Los Angeles Clippers,L.A. Clippers,LAC,-1,Los Angeles Lakers,L.A. Lakers,LAL,-1,...,-7.0,-114.0,7.0,-113.0,-2500.0,1150.0,219.5,117.0,-152.0,2019-20
2,2019-10-23,Final,Charlotte Hornets,Charlotte,CHA,-1,Chicago Bulls,Chicago,CHI,-1,...,4.5,-112.0,-4.5,-120.0,-175.0,132.0,255.5,-117.0,-114.0,2019-20
3,2019-10-23,Final,Orlando Magic,Orlando,ORL,-1,Cleveland Cavaliers,Cleveland,CLE,-1,...,-14.5,-117.0,14.5,-114.0,-670.0,420.0,182.5,-120.0,-112.0,2019-20
4,2019-10-23,Final,Indiana Pacers,Indiana,IND,-1,Detroit Pistons,Detroit,DET,-1,...,1.5,-113.0,-1.5,-118.0,115.0,-150.0,223.0,-143.0,108.0,2019-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6563,2024-12-04,Final,Boston Celtics,Boston,BOS,-1,Detroit Pistons,Detroit,DET,-1,...,-12.5,-108.0,12.5,-112.0,-800.0,550.0,221.5,-110.0,-110.0,2024-25
6564,2024-12-04,Final,Brooklyn Nets,Brooklyn,BKN,-1,Indiana Pacers,Indiana,IND,-1,...,3.0,-105.0,-3.0,-115.0,130.0,-155.0,226.0,-108.0,-112.0,2024-25
6565,2024-12-04,Final,Philadelphia 76ers,Philadelphia,PHI,-1,Orlando Magic,Orlando,ORL,-1,...,5.5,-105.0,-5.5,-115.0,200.0,-245.0,209.5,-110.0,-110.0,2024-25
6566,2024-12-04,Final,Milwaukee Bucks,Milwaukee,MIL,-1,Atlanta Hawks,Atlanta,ATL,-1,...,-4.5,-125.0,4.5,-105.0,-215.0,165.0,236.5,-120.0,-110.0,2024-25


In [42]:
results_df2 = pd.merge(results_df.loc[results_df['Home']==1], results_df.loc[results_df['Home']==0, ['GAME_ID', 'Actual_Score', 'Predicted_Score']]
         , on='GAME_ID', suffixes=['_Home', '_Away'])
results_df2 = results_df2.rename(columns={'Team':'HOME_TEAM_ABBREVIATION', 'Opponent':'AWAY_TEAM_ABBREVIATION'})

results_df2['GAME_DATE'] = pd.to_datetime(results_df2['GAME_DATE'])
# # results_df2 = results_df2[['HOME_TEAM_ABBREVIATION', 'AWAY_TEAM_ABBREVIATION', 'GAME_DATE', 'GAME_ID', 'Matchup', 'Home', 'Predicted_Score_Home', 'Predicted_Score_Away', 'Actual_Score_Home', 'Actual_Score_Away',
# #        ]]


results_df2 = pd.merge(results_df2, betting_data_dk, how='left', on = ['GAME_DATE', 'HOME_TEAM_ABBREVIATION'])

results_df2.columns

Index(['HOME_TEAM_ABBREVIATION', 'AWAY_TEAM_ABBREVIATION_x', 'GAME_DATE',
       'GAME_ID', 'Matchup', 'Home', 'Actual_Score_Home',
       'Predicted_Score_Home', 'Actual_Score_Away', 'Predicted_Score_Away',
       'status', 'home_team', 'home_team_loc', 'home_team_rank', 'away_team',
       'away_team_loc', 'AWAY_TEAM_ABBREVIATION_y', 'away_team_rank',
       'home_score', 'away_score', 'home_spread_draftkings',
       'home_spread_odds_draftkings', 'away_spread_draftkings',
       'away_spread_odds_draftkings', 'home_ml_draftkings',
       'away_ml_draftkings', 'total_draftkings', 'over_odds_draftkings',
       'under_odds_draftkings', 'SEASON'],
      dtype='object')

In [43]:
import pandas as pd
import numpy as np

def calculate_betting_metrics(df):
    """Calculate base metrics for spread, moneyline, and total betting performance."""
    results = {}
    
    # Spread Performance
    df['predicted_spread'] = df['Predicted_Score_Home'] - df['Predicted_Score_Away']
    df['actual_spread'] = df['Actual_Score_Home'] - df['Actual_Score_Away']
    df['spread_bet_result'] = np.where(
        df['predicted_spread'] - df['home_spread_draftkings'] > 0,
        np.where(df['actual_spread'] - df['home_spread_draftkings'] > 0, 1,
                np.where(df['actual_spread'] - df['home_spread_draftkings'] == 0, 0, -1)),
        np.where(df['actual_spread'] - df['home_spread_draftkings'] < 0, 1,
                np.where(df['actual_spread'] - df['home_spread_draftkings'] == 0, 0, -1))
    )
    
    # Moneyline Performance
    df['predicted_winner'] = np.where(df['Predicted_Score_Home'] > df['Predicted_Score_Away'], 1, -1)
    df['actual_winner'] = np.where(df['Actual_Score_Home'] > df['Actual_Score_Away'], 1, -1)
    df['ml_bet_result'] = np.where(df['predicted_winner'] == df['actual_winner'], 1, -1)
    
    # Total Performance
    df['predicted_total'] = df['Predicted_Score_Home'] + df['Predicted_Score_Away']
    df['actual_total'] = df['Actual_Score_Home'] + df['Actual_Score_Away']
    df['total_bet_result'] = np.where(
        df['predicted_total'] > df['total_draftkings'],
        np.where(df['actual_total'] > df['total_draftkings'], 1,
                np.where(df['actual_total'] == df['total_draftkings'], 0, -1)),
        np.where(df['actual_total'] < df['total_draftkings'], 1,
                np.where(df['actual_total'] == df['total_draftkings'], 0, -1))
    )
    
    # Calculate win rates (excluding pushes)
    for bet_type in ['spread', 'ml', 'total']:
        bet_results = df[f'{bet_type}_bet_result']
        wins = sum(bet_results == 1)
        losses = sum(bet_results == -1)
        pushes = sum(bet_results == 0)
        total_decisions = wins + losses
        
        results[f'{bet_type}_win_rate'] = wins / total_decisions if total_decisions > 0 else 0
        results[f'{bet_type}_wins'] = wins
        results[f'{bet_type}_losses'] = losses
        results[f'{bet_type}_pushes'] = pushes
    
    return results

def simulate_flat_betting(df, initial_bankroll=10000, bet_size=100):
    """Simulate flat betting strategy."""
    results = {}
    
    for bet_type in ['spread', 'ml', 'total']:
        bankroll = initial_bankroll
        bets_placed = []
        
        for idx, row in df.iterrows():
            if bet_type == 'spread':
                odds = row['home_spread_odds_draftkings'] if row['predicted_spread'] > row['home_spread_draftkings'] else row['away_spread_odds_draftkings']
            elif bet_type == 'ml':
                odds = row['home_ml_draftkings'] if row['predicted_winner'] == 1 else row['away_ml_draftkings']
            else:  # total
                odds = row['over_odds_draftkings'] if row['predicted_total'] > row['total_draftkings'] else row['under_odds_draftkings']
            
            # Convert American odds to decimal
            if odds > 0:
                decimal_odds = (odds / 100) + 1
            else:
                decimal_odds = (100 / abs(odds)) + 1
            
            result = row[f'{bet_type}_bet_result']
            
            if result != 0:  # Skip pushes
                if result == 1:
                    bankroll += bet_size * (decimal_odds - 1)
                else:
                    bankroll -= bet_size
                
                bets_placed.append(bankroll)
        
        results[f'{bet_type}_final_bankroll'] = bankroll
        results[f'{bet_type}_roi'] = ((bankroll - initial_bankroll) / initial_bankroll) * 100
        results[f'{bet_type}_bankroll_history'] = bets_placed
    
    return results

def kelly_criterion_betting(df, initial_bankroll=10000):
    """Simulate Kelly Criterion betting strategy."""
    results = {}
    
    for bet_type in ['spread', 'ml', 'total']:
        bankroll = initial_bankroll
        bets_placed = []
        
        for idx, row in df.iterrows():
            if bet_type == 'spread':
                odds = row['home_spread_odds_draftkings'] if row['predicted_spread'] > row['home_spread_draftkings'] else row['away_spread_odds_draftkings']
            elif bet_type == 'ml':
                odds = row['home_ml_draftkings'] if row['predicted_winner'] == 1 else row['away_ml_draftkings']
            else:  # total
                odds = row['over_odds_draftkings'] if row['predicted_total'] > row['total_draftkings'] else row['under_odds_draftkings']
            
            # Convert American odds to decimal
            if odds > 0:
                decimal_odds = (odds / 100) + 1
            else:
                decimal_odds = (100 / abs(odds)) + 1
            
            # Calculate Kelly fraction
            win_prob = 0.5  # Using conservative estimate
            b = decimal_odds - 1
            q = 1 - win_prob
            kelly_fraction = (b * win_prob - q) / b
            
            # Limit bet size to 5% of bankroll maximum
            bet_size = min(kelly_fraction * bankroll, 0.05 * bankroll)
            
            result = row[f'{bet_type}_bet_result']
            
            if result != 0:  # Skip pushes
                if result == 1:
                    bankroll += bet_size * (decimal_odds - 1)
                else:
                    bankroll -= bet_size
                
                bets_placed.append(bankroll)
        
        results[f'{bet_type}_final_bankroll'] = bankroll
        results[f'{bet_type}_roi'] = ((bankroll - initial_bankroll) / initial_bankroll) * 100
        results[f'{bet_type}_bankroll_history'] = bets_placed
    
    return results

def percentage_bankroll_betting(df, initial_bankroll=10000, percentage=0.01):
    """Simulate betting a fixed percentage of current bankroll."""
    results = {}
    
    for bet_type in ['spread', 'ml', 'total']:
        bankroll = initial_bankroll
        bets_placed = []
        
        for idx, row in df.iterrows():
            if bet_type == 'spread':
                odds = row['home_spread_odds_draftkings'] if row['predicted_spread'] > row['home_spread_draftkings'] else row['away_spread_odds_draftkings']
            elif bet_type == 'ml':
                odds = row['home_ml_draftkings'] if row['predicted_winner'] == 1 else row['away_ml_draftkings']
            else:  # total
                odds = row['over_odds_draftkings'] if row['predicted_total'] > row['total_draftkings'] else row['under_odds_draftkings']
            
            # Convert American odds to decimal
            if odds > 0:
                decimal_odds = (odds / 100) + 1
            else:
                decimal_odds = (100 / abs(odds)) + 1
            
            bet_size = bankroll * percentage
            result = row[f'{bet_type}_bet_result']
            
            if result != 0:  # Skip pushes
                if result == 1:
                    bankroll += bet_size * (decimal_odds - 1)
                else:
                    bankroll -= bet_size
                
                bets_placed.append(bankroll)
        
        results[f'{bet_type}_final_bankroll'] = bankroll
        results[f'{bet_type}_roi'] = ((bankroll - initial_bankroll) / initial_bankroll) * 100
        results[f'{bet_type}_bankroll_history'] = bets_placed
    
    return results

def analyze_betting_performance(df):
    """Comprehensive analysis of betting performance with different strategies."""
    # Calculate base metrics
    base_metrics = calculate_betting_metrics(df)
    
    # Simulate different betting strategies
    flat_betting_results = simulate_flat_betting(df)
    kelly_results = kelly_criterion_betting(df)
    percentage_results = percentage_bankroll_betting(df)
    
    return {
        'base_metrics': base_metrics,
        'flat_betting': flat_betting_results,
        'kelly_criterion': kelly_results,
        'percentage_bankroll': percentage_results
    }

In [44]:
analyze_betting_performance(results_df2)

{'base_metrics': {'spread_win_rate': 0.778816199376947,
  'spread_wins': 250,
  'spread_losses': 71,
  'spread_pushes': 2,
  'ml_win_rate': 0.6346749226006192,
  'ml_wins': 205,
  'ml_losses': 118,
  'ml_pushes': 0,
  'total_win_rate': 0.48286604361370716,
  'total_wins': 155,
  'total_losses': 166,
  'total_pushes': 2},
 'flat_betting': {'spread_final_bankroll': 25534.867773866416,
  'spread_roi': 155.34867773866415,
  'spread_bankroll_history': [10089.285714285714,
   9989.285714285714,
   10081.878306878307,
   10177.116402116402,
   10077.116402116402,
   10164.072923855532,
   10253.358638141246,
   10333.358638141246,
   10416.69197147458,
   10316.69197147458,
   10411.930066712675,
   10495.26340004601,
   10584.549114331723,
   10673.834828617437,
   10766.42742121003,
   10859.020013802623,
   10949.929104711713,
   10849.929104711713,
   10940.838195620803,
   11030.123909906517,
   11122.71650249911,
   11213.6255934082,
   11113.6255934082,
   11204.53468431729,
   11293.8

In [45]:
def analyze_spread_wins(df):
    """
    Analyze and display games where the model predicted spread wins.
    Includes detailed calculations to verify spread win/loss determination.
    """
    # Calculate predicted and actual spread differentials
    df['predicted_spread'] = df['Predicted_Score_Home'] - df['Predicted_Score_Away']
    df['actual_spread'] = df['Actual_Score_Home'] - df['Actual_Score_Away']
    
    # Determine bet direction (home or away) based on predicted spread vs line
    df['bet_on_home'] = df['predicted_spread'] > df['home_spread_draftkings']
    
    # Calculate if the bet won
    df['spread_bet_result'] = np.where(
        df['bet_on_home'],
        # If bet on home
        np.where(df['actual_spread'] > df['home_spread_draftkings'], 'WIN',
                np.where(df['actual_spread'] == df['home_spread_draftkings'], 'PUSH', 'LOSS')),
        # If bet on away
        np.where(df['actual_spread'] < df['home_spread_draftkings'], 'WIN',
                np.where(df['actual_spread'] == df['home_spread_draftkings'], 'PUSH', 'LOSS'))
    )
    
    # Create a clear analysis DataFrame
    analysis_df = df[[
        'GAME_DATE', 
        'HOME_TEAM_ABBREVIATION',
        'AWAY_TEAM_ABBREVIATION_x',
        'home_spread_draftkings',
        'Predicted_Score_Home',
        'Predicted_Score_Away',
        'predicted_spread',
        'Actual_Score_Home',
        'Actual_Score_Away',
        'actual_spread',
        'bet_on_home',
        'spread_bet_result'
    ]].copy()
    
    # Add explanation column
    analysis_df['explanation'] = analysis_df.apply(
        lambda row: (
            f"Predicted {row['HOME_TEAM_ABBREVIATION'] if row['bet_on_home'] else row['AWAY_TEAM_ABBREVIATION_x']} "
            f"to cover {abs(row['home_spread_draftkings'])} points. "
            f"Actual margin: {abs(row['actual_spread'])} points."
        ),
        axis=1
    )
    
    # Sort by date
    analysis_df = analysis_df.sort_values('GAME_DATE')
    
    # Calculate summary statistics
    total_bets = len(analysis_df)
    wins = (analysis_df['spread_bet_result'] == 'WIN').sum()
    losses = (analysis_df['spread_bet_result'] == 'LOSS').sum()
    pushes = (analysis_df['spread_bet_result'] == 'PUSH').sum()
    win_pct = wins / (wins + losses) if (wins + losses) > 0 else 0
    
    print(f"\nSpread Betting Summary:")
    print(f"Total Bets: {total_bets}")
    print(f"Wins: {wins}")
    print(f"Losses: {losses}")
    print(f"Pushes: {pushes}")
    print(f"Win Percentage: {win_pct:.2%}")
    
    return analysis_df

def display_spread_wins(df):
    """Display only the winning spread bets with detailed information."""
    wins_df = df[df['spread_bet_result'] == 'WIN'].copy()
    
    # Format for display
    display_df = wins_df[[
        'GAME_DATE',
        'HOME_TEAM_ABBREVIATION',
        'AWAY_TEAM_ABBREVIATION_x',
        'home_spread_draftkings',
        'Actual_Score_Home',
        'Actual_Score_Away',
        'actual_spread',
        'Predicted_Score_Home',
        'Predicted_Score_Away',
        'predicted_spread',
        'bet_on_home',
        'explanation'
    ]]
    
    return display_df

# Function to verify a specific game's spread result
def verify_spread_result(row):
    """Detailed verification of a single game's spread result."""
    print(f"\nGame: {row['AWAY_TEAM_ABBREVIATION_x']} @ {row['HOME_TEAM_ABBREVIATION']} on {row['GAME_DATE']}")
    print(f"Spread: {row['HOME_TEAM_ABBREVIATION']} {row['home_spread_draftkings']}")
    print(f"Final Score: {row['HOME_TEAM_ABBREVIATION']} {row['Actual_Score_Home']} - {row['AWAY_TEAM_ABBREVIATION_x']} {row['Actual_Score_Away']}")
    print(f"Actual Margin: {row['actual_spread']}")
    print(f"Bet on: {'Home' if row['bet_on_home'] else 'Away'}")
    print(f"Result: {row['spread_bet_result']}")
    print(f"Explanation: {row['explanation']}")
    
    
    
# Get full analysis
analysis_df = analyze_spread_wins(results_df2)

# See only the winning bets
winning_bets = display_spread_wins(analysis_df)
print("\nWinning Spread Bets:")
winning_bets




Spread Betting Summary:
Total Bets: 323
Wins: 250
Losses: 71
Pushes: 2
Win Percentage: 77.88%

Winning Spread Bets:


,GAME_DATE,HOME_TEAM_ABBREVIATION,AWAY_TEAM_ABBREVIATION_x,home_spread_draftkings,Actual_Score_Home,Actual_Score_Away,actual_spread,Predicted_Score_Home,Predicted_Score_Away,predicted_spread,bet_on_home,explanation
0,2024-10-22,BOS,NYK,-6.0,132,109,23,114.362900,106.812332,7.550568,True,Predicted BOS to cover 6.0 points. Actual margin: 23 points.
11,2024-10-23,POR,GSW,7.5,104,140,-36,106.144287,116.383911,-10.239624,False,Predicted GSW to cover 7.5 points. Actual margin: 36 points.
8,2024-10-23,NOP,CHI,-4.5,123,111,12,108.887619,109.723251,-0.835632,True,Predicted NOP to cover 4.5 points. Actual margin: 12 points.
7,2024-10-23,HOU,CHA,-8.5,105,110,-5,116.139290,108.496498,7.642792,True,Predicted HOU to cover 8.5 points. Actual margin: 5 points.
10,2024-10-23,LAC,PHX,5.5,113,116,-3,108.548637,109.964493,-1.415855,False,Predicted PHX to cover 5.5 points. Actual margin: 3 points.
...,...,...,...,...,...,...,...,...,...,...,...,...
316,2024-12-03,LAC,POR,-8.5,127,105,22,113.711098,104.512329,9.198769,True,Predicted LAC to cover 8.5 points. Actual margin: 22 points.
320,2024-12-04,PHI,ORL,5.5,102,106,-4,101.116135,107.146591,-6.030457,False,Predicted ORL to cover 5.5 points. Actual margin: 4 points.
319,2024-12-04,MIA,LAL,-3.5,134,93,41,109.695541,109.121696,0.573845,True,Predicted MIA to cover 3.5 points. Actual margin: 41 points.
317,2024-12-04,BOS,DET,-12.5,130,120,10,116.038849,105.317352,10.721497,True,Predicted BOS to cover 12.5 points. Actual margin: 10 points.


In [46]:
def transform_betting_results(analysis_results):
    """
    Transform betting analysis results into the format needed for visualization.
    
    Args:
        analysis_results (dict): Results from analyze_betting_performance function
        
    Returns:
        dict: Formatted data ready for visualization
    """
    base_metrics = analysis_results['base_metrics']
    flat_betting = analysis_results['flat_betting']
    kelly_criterion = analysis_results['kelly_criterion']
    percentage_bankroll = analysis_results['percentage_bankroll']
    
    # Format win rate data
    win_rate_data = [
        {
            'name': 'Spread',
            'winRate': base_metrics['spread_win_rate'],
            'color': '#60a5fa'
        },
        {
            'name': 'Moneyline',
            'winRate': base_metrics['ml_win_rate'],
            'color': '#34d399'
        },
        {
            'name': 'Total',
            'winRate': base_metrics['total_win_rate'],
            'color': '#f87171'
        }
    ]
    
    # Format ROI data
    roi_data = [
        {
            'name': 'Flat Betting',
            'spread': flat_betting['spread_roi'],
            'moneyline': flat_betting['ml_roi'],
            'total': flat_betting['total_roi']
        },
        {
            'name': 'Kelly Criterion',
            'spread': kelly_criterion['spread_roi'],
            'moneyline': kelly_criterion['ml_roi'],
            'total': kelly_criterion['total_roi']
        },
        {
            'name': 'Percentage',
            'spread': percentage_bankroll['spread_roi'],
            'moneyline': percentage_bankroll['ml_roi'],
            'total': percentage_bankroll['total_roi']
        }
    ]
    
    # Format bankroll evolution data
    # Get the length of the shortest history to align the data
    min_length = min(
        len(flat_betting['spread_bankroll_history']),
        len(kelly_criterion['spread_bankroll_history']),
        len(percentage_bankroll['spread_bankroll_history'])
    )
    
    bankroll_data = []
    for i in range(min_length):
        bankroll_data.append({
            'bet': i + 1,
            'flat': flat_betting['spread_bankroll_history'][i],
            'kelly': kelly_criterion['spread_bankroll_history'][i],
            'percentage': percentage_bankroll['spread_bankroll_history'][i]
        })
    
    # Additional summary statistics
    summary_stats = {
        'total_bets': {
            'spread': base_metrics['spread_wins'] + base_metrics['spread_losses'],
            'moneyline': base_metrics['ml_wins'] + base_metrics['ml_losses'],
            'total': base_metrics['total_wins'] + base_metrics['total_losses']
        },
        'pushes': {
            'spread': base_metrics['spread_pushes'],
            'moneyline': base_metrics['ml_pushes'],
            'total': base_metrics['total_pushes']
        },
        'final_bankrolls': {
            'flat': {
                'spread': flat_betting['spread_final_bankroll'],
                'moneyline': flat_betting['ml_final_bankroll'],
                'total': flat_betting['total_final_bankroll']
            },
            'kelly': {
                'spread': kelly_criterion['spread_final_bankroll'],
                'moneyline': kelly_criterion['ml_final_bankroll'],
                'total': kelly_criterion['total_final_bankroll']
            },
            'percentage': {
                'spread': percentage_bankroll['spread_final_bankroll'],
                'moneyline': percentage_bankroll['ml_final_bankroll'],
                'total': percentage_bankroll['total_final_bankroll']
            }
        }
    }
    
    return {
        'winRateData': win_rate_data,
        'roiData': roi_data,
        'bankrollData': bankroll_data,
        'summaryStats': summary_stats
    }

In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

def plot_win_rates(analysis_results):
    """Create a bar plot of win rates by bet type using Plotly."""
    base_metrics = analysis_results['base_metrics']
    
    # Prepare data
    bet_types = ['Spread', 'Moneyline', 'Total']
    win_rates = [
        base_metrics['spread_win_rate'],
        base_metrics['ml_win_rate'],
        base_metrics['total_win_rate']
    ]
    
    # Create bar plot
    fig = go.Figure(data=[
        go.Bar(
            x=bet_types,
            y=win_rates,
            text=[f'{rate:.1%}' for rate in win_rates],
            textposition='auto',
        )
    ])
    
    fig.update_layout(
        title='Win Rates by Bet Type',
        yaxis_title='Win Rate',
        yaxis_tickformat=',.0%',
        showlegend=False,
        plot_bgcolor='white'
    )
    
    return fig

def plot_roi_comparison(analysis_results):
    """Create a grouped bar plot of ROI by strategy and bet type using Plotly."""
    strategies = {
        'Flat Betting': analysis_results['flat_betting'],
        'Kelly Criterion': analysis_results['kelly_criterion'],
        'Percentage Bankroll': analysis_results['percentage_bankroll']
    }
    
    fig = go.Figure()
    
    bet_types = ['spread', 'ml', 'total']
    bet_labels = ['Spread', 'Moneyline', 'Total']
    colors = ['rgb(99, 110, 250)', 'rgb(239, 85, 59)', 'rgb(0, 204, 150)']
    
    for bet_type, label, color in zip(bet_types, bet_labels, colors):
        roi_values = [strategies[strat][f'{bet_type}_roi'] for strat in strategies.keys()]
        
        fig.add_trace(go.Bar(
            name=label,
            x=list(strategies.keys()),
            y=roi_values,
            text=[f'{roi:.1f}%' for roi in roi_values],
            textposition='auto',
            marker_color=color
        ))
    
    fig.update_layout(
        title='ROI by Strategy and Bet Type',
        yaxis_title='ROI (%)',
        barmode='group',
        plot_bgcolor='white'
    )
    
    return fig

def plot_bankroll_evolution(analysis_results):
    """Create a line plot of bankroll evolution using Plotly."""
    strategies = {
        'Flat Betting': analysis_results['flat_betting'],
        'Kelly Criterion': analysis_results['kelly_criterion'],
        'Percentage Bankroll': analysis_results['percentage_bankroll']
    }
    
    fig = go.Figure()
    colors = ['rgb(99, 110, 250)', 'rgb(239, 85, 59)', 'rgb(0, 204, 150)']
    
    for (strat_name, strat_data), color in zip(strategies.items(), colors):
        for bet_type in ['spread', 'ml', 'total']:
            history = strat_data[f'{bet_type}_bankroll_history']
            
            fig.add_trace(go.Scatter(
                x=list(range(1, len(history) + 1)),
                y=history,
                name=f'{strat_name} - {bet_type.upper()}',
                line=dict(color=color, dash='solid' if bet_type == 'spread' else 'dash' if bet_type == 'ml' else 'dot')
            ))
    
    fig.update_layout(
        title='Bankroll Evolution by Strategy and Bet Type',
        xaxis_title='Bet Number',
        yaxis_title='Bankroll ($)',
        yaxis_tickformat='$,.0f',
        plot_bgcolor='white'
    )
    
    return fig

def create_summary_table(analysis_results):
    """Create a summary table of betting performance using Plotly."""
    base_metrics = analysis_results['base_metrics']
    
    # Prepare data
    data = {
        'Metric': [
            'Total Bets',
            'Wins',
            'Losses',
            'Pushes',
            'Win Rate',
            'Best Strategy ROI'
        ],
        'Spread': [
            base_metrics['spread_wins'] + base_metrics['spread_losses'],
            base_metrics['spread_wins'],
            base_metrics['spread_losses'],
            base_metrics['spread_pushes'],
            f"{base_metrics['spread_win_rate']:.1%}",
            f"{max(analysis_results['flat_betting']['spread_roi'], analysis_results['kelly_criterion']['spread_roi'], analysis_results['percentage_bankroll']['spread_roi']):.1f}%"
        ],
        'Moneyline': [
            base_metrics['ml_wins'] + base_metrics['ml_losses'],
            base_metrics['ml_wins'],
            base_metrics['ml_losses'],
            base_metrics['ml_pushes'],
            f"{base_metrics['ml_win_rate']:.1%}",
            f"{max(analysis_results['flat_betting']['ml_roi'], analysis_results['kelly_criterion']['ml_roi'], analysis_results['percentage_bankroll']['ml_roi']):.1f}%"
        ],
        'Total': [
            base_metrics['total_wins'] + base_metrics['total_losses'],
            base_metrics['total_wins'],
            base_metrics['total_losses'],
            base_metrics['total_pushes'],
            f"{base_metrics['total_win_rate']:.1%}",
            f"{max(analysis_results['flat_betting']['total_roi'], analysis_results['kelly_criterion']['total_roi'], analysis_results['percentage_bankroll']['total_roi']):.1f}%"
        ]
    }
    
    fig = go.Figure(data=[go.Table(
        header=dict(
            values=list(data.keys()),
            fill_color='paleturquoise',
            align='left'
        ),
        cells=dict(
            values=list(data.values()),
            fill_color='lavender',
            align='left'
        )
    )])
    
    fig.update_layout(
        title='Betting Performance Summary'
    )
    
    return fig

def visualize_all_results(analysis_results):
    """Create and display all visualizations."""
    # Create all plots
    win_rates_fig = plot_win_rates(analysis_results)
    roi_fig = plot_roi_comparison(analysis_results)
    bankroll_fig = plot_bankroll_evolution(analysis_results)
    summary_table = create_summary_table(analysis_results)
    
    # Display all plots
    win_rates_fig.show()
    roi_fig.show()
    bankroll_fig.show()
    summary_table.show()
    
    return {
        'win_rates': win_rates_fig,
        'roi': roi_fig,
        'bankroll': bankroll_fig,
        'summary': summary_table
    }

In [48]:
betting_results = analyze_betting_performance(results_df2)

visualize_all_results(betting_results)

{'win_rates': Figure({
     'data': [{'text': [77.9%, 63.5%, 48.3%],
               'textposition': 'auto',
               'type': 'bar',
               'x': [Spread, Moneyline, Total],
               'y': [0.778816199376947, 0.6346749226006192, 0.48286604361370716]}],
     'layout': {'plot_bgcolor': 'white',
                'showlegend': False,
                'template': '...',
                'title': {'text': 'Win Rates by Bet Type'},
                'yaxis': {'tickformat': ',.0%', 'title': {'text': 'Win Rate'}}}
 }),
 'roi': Figure({
     'data': [{'marker': {'color': 'rgb(99, 110, 250)'},
               'name': 'Spread',
               'text': [155.3%, -100.0%, 366.3%],
               'textposition': 'auto',
               'type': 'bar',
               'x': [Flat Betting, Kelly Criterion, Percentage Bankroll],
               'y': [155.34867773866415, -99.97655101373674, 366.3271740762139]},
              {'marker': {'color': 'rgb(239, 85, 59)'},
               'name': 'Moneyline'